In [4]:
print("Hello")

Hello


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import EfficientNetB0
from sklearn.utils.class_weight import compute_class_weight

# ------------------------------
# CONFIG
# ------------------------------
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 50  # you can increase if needed

DATA_DIR = r"C:\Users\SHISHIR\Desktop\EPICS\Skin cancer ISIC The International Skin Imaging Collaboration"

# ------------------------------
# DATA GENERATORS
# ------------------------------
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1  # split for validation
)

train_gen = train_datagen.flow_from_directory(
    os.path.join(DATA_DIR, "train"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    color_mode='rgb',
    subset='training',
    shuffle=True
)

val_gen = train_datagen.flow_from_directory(
    os.path.join(DATA_DIR, "train"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    color_mode='rgb',
    subset='validation',
    shuffle=False
)

NUM_CLASSES = len(train_gen.class_indices)
print("Detected classes:", train_gen.class_indices)

# ------------------------------
# CLASS WEIGHTS
# ------------------------------
y_train = train_gen.classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights))
print("Class weights:", class_weights)

# ------------------------------
# MODEL (EfficientNetB0 from scratch)
# ------------------------------
base_model = EfficientNetB0(
    weights=None,  # train from scratch
    include_top=False,
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ------------------------------
# CALLBACKS
# ------------------------------
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_accuracy', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1)

# ------------------------------
# TRAINING
# ------------------------------
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    class_weight=class_weights,
    callbacks=[checkpoint, early_stop]
)

# ------------------------------
# EVALUATION
# ------------------------------
test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    os.path.join(DATA_DIR, "test"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=False
)

loss, acc = model.evaluate(test_gen)
print(f"✅ Test Accuracy: {acc*100:.2f}%")



Found 2020 images belonging to 9 classes.
Found 219 images belonging to 9 classes.
Detected classes: {'actinic keratosis': 0, 'basal cell carcinoma': 1, 'dermatofibroma': 2, 'melanoma': 3, 'nevus': 4, 'pigmented benign keratosis': 5, 'seborrheic keratosis': 6, 'squamous cell carcinoma': 7, 'vascular lesion': 8}
Class weights: {0: np.float64(2.179072276159655), 1: np.float64(0.6620780072107506), 2: np.float64(2.6098191214470283), 3: np.float64(0.5682137834036568), 4: np.float64(0.6970324361628709), 5: np.float64(0.5395299145299145), 6: np.float64(3.2063492063492065), 7: np.float64(1.3769597818677572), 8: np.float64(1.781305114638448)}


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling_2 (Rescaling)       │ (None, 224, 224, 3)       │               0 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ normalization_1               │ (None, 224, 224, 3)       │               7 │ rescaling_2[0][0]          │
│ (Normalization)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv_pad (ZeroPadding2D) │ (None, 225, 225, 3)       │               0 │ normalization_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv (Conv2D)            │ (None, 112, 112, 32)      │             864 │ stem_conv_pad[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_bn (BatchNormalization)  │ (None, 112, 112, 32)      │             128 │ stem_conv[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_activation (Activation)  │ (None, 112, 112, 32)      │               0 │ stem_bn[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_dwconv                │ (None, 112, 112, 32)      │             288 │ stem_activation[0][0]      │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_bn                    │ (None, 112, 112, 32)      │             128 │ block1a_dwconv[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_activation            │ (None, 112, 112, 32)      │               0 │ block1a_bn[0][0]           │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_squeeze            │ (None, 32)                │               0 │ block1a_activation[0][0]   │
│ (GlobalAveragePooling2D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reshape (Reshape)  │ (None, 1, 1, 32)          │               0 │ block1a_se_squeeze[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reduce (Conv2D)    │ (None, 1, 1, 8)           │             264 │ block1a_se_reshape[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_expand (Conv2D)    │ (None, 1, 1, 32)          │             288 │ block1a_se_reduce[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_excite (Multiply)  │ (None, 112, 112, 32)      │               

 Total params: 4,379,820 (16.71 MB)

 Trainable params: 4,337,797 (16.55 MB)

...

127/127 ━━━━━━━━━━━━━━━━━━━━ 166s 1s/step - accuracy: 0.2634 - loss: 1.7056 - val_accuracy: 0.2648 - val_loss: 2.1921
Epoch 8/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2975 - loss: 1.6392
Epoch 8: val_accuracy did not improve from 0.26484
127/127 ━━━━━━━━━━━━━━━━━━━━ 184s 1s/step - accuracy: 0.2886 - loss: 1.6770 - val_accuracy: 0.2009 - val_loss: 2.4201
Epoch 9/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2999 - loss: 1.6554
Epoch 9: val_accuracy did not improve from 0.26484
127/127 ━━━━━━━━━━━━━━━━━━━━ 179s 1s/step - accuracy: 0.3064 - loss: 1.6812 - val_accuracy: 0.2420 - val_loss: 3.1316
Epoch 10/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2966 - loss: 1.6443
Epoch 10: val_accuracy did not improve from 0.26484
127/127 ━━━━━━━━━━━━━━━━━━━━ 183s 1s/step - accuracy: 0.2881 - loss: 1.6647 - val_accuracy: 0.2420 - val_loss: 2.4002
Epoch 11/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3043 - loss: 1.5867
Epoch 11: val_accuracy did not i

127/127 ━━━━━━━━━━━━━━━━━━━━ 192s 2s/step - accuracy: 0.3213 - loss: 1.6046 - val_accuracy: 0.3014 - val_loss: 1.9721
Epoch 14/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3468 - loss: 1.6083
Epoch 14: val_accuracy improved from 0.30137 to 0.31963, saving model to best_model.h5


127/127 ━━━━━━━━━━━━━━━━━━━━ 193s 2s/step - accuracy: 0.3376 - loss: 1.5898 - val_accuracy: 0.3196 - val_loss: 1.6581
Epoch 15/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3667 - loss: 1.5202
Epoch 15: val_accuracy did not improve from 0.31963
127/127 ━━━━━━━━━━━━━━━━━━━━ 180s 1s/step - accuracy: 0.3653 - loss: 1.5635 - val_accuracy: 0.1553 - val_loss: 4.7886
Epoch 16/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3616 - loss: 1.5615
Epoch 16: val_accuracy did not improve from 0.31963
127/127 ━━━━━━━━━━━━━━━━━━━━ 173s 1s/step - accuracy: 0.3619 - loss: 1.5659 - val_accuracy: 0.2922 - val_loss: 1.8507
Epoch 17/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3678 - loss: 1.5261
Epoch 17: val_accuracy improved from 0.31963 to 0.34247, saving model to best_model.h5


127/127 ━━━━━━━━━━━━━━━━━━━━ 198s 2s/step - accuracy: 0.3564 - loss: 1.5371 - val_accuracy: 0.3425 - val_loss: 2.2490
Epoch 18/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3743 - loss: 1.5399
Epoch 18: val_accuracy did not improve from 0.34247
127/127 ━━━━━━━━━━━━━━━━━━━━ 200s 2s/step - accuracy: 0.3728 - loss: 1.5299 - val_accuracy: 0.3242 - val_loss: 1.7355
Epoch 19/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3745 - loss: 1.4880
Epoch 19: val_accuracy did not improve from 0.34247
127/127 ━━━━━━━━━━━━━━━━━━━━ 185s 1s/step - accuracy: 0.3822 - loss: 1.4807 - val_accuracy: 0.3333 - val_loss: 1.9530
Epoch 20/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3774 - loss: 1.4758
Epoch 20: val_accuracy did not improve from 0.34247
127/127 ━━━━━━━━━━━━━━━━━━━━ 173s 1s/step - accuracy: 0.3960 - loss: 1.4428 - val_accuracy: 0.2877 - val_loss: 1.8890
Epoch 21/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3703 - loss: 1.4722
Epoch 21: val_accuracy did n

127/127 ━━━━━━━━━━━━━━━━━━━━ 196s 2s/step - accuracy: 0.3772 - loss: 1.4440 - val_accuracy: 0.4110 - val_loss: 1.5279
Epoch 23/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3370 - loss: 1.4195
Epoch 23: val_accuracy did not improve from 0.41096
127/127 ━━━━━━━━━━━━━━━━━━━━ 176s 1s/step - accuracy: 0.3396 - loss: 1.4340 - val_accuracy: 0.3242 - val_loss: 1.9064
Epoch 24/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4064 - loss: 1.4055
Epoch 24: val_accuracy did not improve from 0.41096
127/127 ━━━━━━━━━━━━━━━━━━━━ 189s 1s/step - accuracy: 0.3817 - loss: 1.4457 - val_accuracy: 0.3653 - val_loss: 1.7034
Epoch 25/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3795 - loss: 1.4478
Epoch 25: val_accuracy did not improve from 0.41096
127/127 ━━━━━━━━━━━━━━━━━━━━ 171s 1s/step - accuracy: 0.3708 - loss: 1.4405 - val_accuracy: 0.2877 - val_loss: 2.9770
Epoch 26/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3925 - loss: 1.3830
Epoch 26: val_accuracy did n

In [3]:
# ------------------------------
# SAVE MODEL
# ------------------------------
model.save("final_model.keras")
print("🎉 Model saved as final_model.keras")

🎉 Model saved as final_model.keras
